In [1]:
import numpy as np
import torch
from pkg_resources import packaging
print("Torch version:", torch.__version__)

Torch version: 1.9.0+cu111


/home/kenan/.conda/envs/clip-torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import clip
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [4]:
model, preprocess = clip.load("ViT-B/32")
model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size
print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)
preprocess

Model parameters: 151,277,313
Input resolution: 224
Context length: 77
Vocab size: 49408


Compose(
    Resize(size=224, interpolation=bicubic, max_size=None, antialias=None)
    CenterCrop(size=(224, 224))
    <function _convert_image_to_rgb at 0x7f5d9266b040>
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
)

In [5]:
import pandas as pd
svo_cleaned = pd.read_csv('svo_valid.csv')
svo_cleaned.head(2)

,sentence,pos_triplet,neg_triplet,pos_url,neg_url,pos_image_id,neg_image_id,subj_neg,verb_neg,obj_neg
0,Girl is standing in the grass.,"girl,stand,grass","dog,stand,grass",https://previews.123rf.com/images/aleksejzhagu...,https://images-na.ssl-images-amazon.com/images...,0,1,True,False,False
1,A woman lies on a beach.,"woman,lie,beach","woman,jump,beach",https://media.istockphoto.com/photos/man-and-w...,https://media.istockphoto.com/photos/woman-to-...,2,3,False,True,False


In [6]:
import os
import skimage
import IPython.display
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

from collections import OrderedDict
import torch
from PIL import UnidentifiedImageError

In [7]:
matched_id = [] ## if cosine similarity to positive image larger: 0, if not: 1
image_path = "/data1/kenan/SVO_Probes/images/"
for idx, row in svo_cleaned.iterrows():
    try:
        text = row.sentence
        image_pos = preprocess(Image.open(os.path.join(skimage.data_dir, image_path+str(row.pos_image_id) + ".jpg")).convert("RGB"))
        image_neg = preprocess(Image.open(os.path.join(skimage.data_dir, image_path+str(row.neg_image_id) + ".jpg")).convert("RGB"))
        images = [image_pos, image_neg]
        image_input = torch.tensor(np.stack(images)).cuda()
        text_tokens = clip.tokenize([text]).cuda()
        with torch.no_grad():
            image_features = model.encode_image(image_input).float()
            text_features = model.encode_text(text_tokens).float()
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_features /= text_features.norm(dim=-1, keepdim=True)
        similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T
        matched_id.append(np.argmax(similarity))
    except UnidentifiedImageError:
        matched_id.append("image_failed")
    

/home/kenan/.conda/envs/clip-torch/lib/python3.9/site-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/home/kenan/.conda/envs/clip-torch/lib/python3.9/site-packages/PIL/TiffImagePlugin.py:845: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 2. 
  warnings.warn(str(msg))


OSError: image file is truncated

In [10]:
print(len(matched_id))
print(len(svo_cleaned))

35763
35763


In [9]:
svo_cleaned = svo_cleaned[:35763]

In [ ]:
sum(matched_id)

In [11]:
svo_cleaned["matched_id"] = matched_id

In [14]:
svo_cleaned = svo_cleaned[svo_cleaned.matched_id != "image_failed"]

In [15]:
sum(list(svo_cleaned["matched_id"]))

6047

In [16]:
len(svo_cleaned)

32938

In [18]:
svo_cleaned.to_csv("clip_result.csv", index = False)

In [20]:
verb = svo_cleaned[svo_cleaned["verb_neg"] == True]

In [21]:
len(verb)

20810

In [23]:
sum(list(verb["matched_id"]))

4382

In [30]:
subject = svo_cleaned[svo_cleaned["subj_neg"] == True]
print(len(subject))
sum(list(subject["matched_id"]))

5252


850

In [29]:
object = svo_cleaned[svo_cleaned["obj_neg"] == True]
print(len(object))
sum(list(object["matched_id"]))

6876


815